# Module 1 - Thu thập dữ liệu từ [nhatot.vn](https://www.nhatot.com/thue-can-ho-chung-cu-tp-ho-chi-minh)

## 1. Mục tiêu

### 1.1. Kịch bản

Thực hiện thu thập dữ liệu chung cư cho thuê ở thành phố Hồ Chí Minh từ [nhatot.vn](https://www.nhatot.com/thue-can-ho-chung-cu-tp-ho-chi-minh), xem xét sơ bộ ta thấy:

+ Mỗi trang sẽ chứa 20 tin rao cho thuê, và có khoảng 1000 trang
+ Thông tin cần phải thu thập cho mỗi căn hộ:

| Column              | Description               |
|---------------------|---------------------------|
| ad_id               | id của tin (để phân biệt) |
| list_id             | id trong danh sách        |
| subject             | Tiêu đề của tin           |
| price               | Giá thuê                  |
| size                | Diện tích                 |
| rooms               | Số phòng                  |
| toilets             | Số nhà vệ sinh            |
| area_name           | Quận huyện                |
| region_name         | Thành phố, tỉnh           |
| ward_name           | Phường                    |
| street_name         | Đuờng                     |
| detail_address      | Địa chỉ chi tiết          |
| zero_deposit        | Có cần tiền cọc không?    |
| escrow_can_deposit  | Cọc bao nhiêu tháng       |
| longitude           | Kinh độ                   |
| latitude            | Vĩ độ                     |

## 2. Thực hiện

### 2.1. Import thư viện cần thiết

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait

import csv
import time
import json

### 2.2. CONSTANT

In [58]:
SLEEP_TIME = 6
BASE_URL='https://www.nhatot.com/thue-can-ho-chung-cu-tp-ho-chi-minh'
LIMIT_PAGE = 1000

### 2.3. Init chrome driver

In [74]:
# Các option cho chrome: ẩn danh, cửa sổ
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36"

options = webdriver.ChromeOptions()
# options.headless = True
options.add_argument(f'user-agent={user_agent}')
options.add_argument("--window-size=1920,1080")
# options.add_argument('--ignore-certificate-errors')
# options.add_argument('--allow-running-insecure-content')
# options.add_argument("--disable-extensions")
# options.add_argument("--proxy-server='direct://'")
# options.add_argument("--proxy-bypass-list=*")
# options.add_argument("--start-maximized")
#
# options.add_argument('--disable-gpu')
# options.add_argument('--disable-dev-shm-usage')
# options.add_argument('--no-sandbox')

# khởi tạo driver
driver = webdriver.Chrome(chrome_options=options, executable_path='./driver/chromedriver')

/var/folders/cf/tjy8lz4n39q0hzxh7gb0n9tr0000gn/T/ipykernel_25715/2067507462.py:20: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options, executable_path='./driver/chromedriver')


### 2.3. Craw data

Cách thực hiện:
    + Do mỗi trang có sẵn một script tag với id '__NEXT_DATA__' chứa thông tin chi tiết của 20 căn hộ trong danh sách, nên ta chỉ cần request tới trang danh sách và lấy dữ liệu mà không cần vào chi trang chi tiết
    + Vì trang web sử dụng CloudFlare security nên với chỉ mở một browser duy nhất, mỗi lần crawl xong dữ liệu cần tìm next button để chuyển qua trang tiếp theo

In [72]:
# get all apartments from 1000 pages
def get_data_per_page(limit):
    all_apartments = []
    page = limit
    # Go to page
    last_page_url = f'{BASE_URL}?page={page}'
    driver.get(last_page_url)
    time.sleep(SLEEP_TIME)
    # Scroll to end of page (for finding next button)
    driver.execute_script("window.scrollBy(0,document.body.scrollHeight)","")
    time.sleep(SLEEP_TIME)
    old_raw = ''

    while page > 1:
        print(f'Get data from page {page}')
        # get raw data from a script tag
        page_source = BeautifulSoup(driver.page_source, "html.parser")
        new_raw = page_source.find('script', id='__NEXT_DATA__').string
        print(old_raw == new_raw)
        # convert raw to dict
        data = json.loads(new_raw)
        # get list apartment
        list_apartment = data['props']['initialState']['adlisting']['data']['ads']
        # add to results
        all_apartments.extend(list_apartment)
        # find next button need to wait page load fully
        back_next_buttons = driver.find_elements_by_class_name("Paging_redirectPageBtn__KvsqJ")
        while len(back_next_buttons) < 2:
            time.sleep(SLEEP_TIME)
            back_next_buttons = driver.find_elements_by_class_name("Paging_redirectPageBtn__KvsqJ")

        back_button = back_next_buttons[0]
        # navigate to next page
        page -= 1
        driver.execute_script("arguments[0].click();", back_button)
        time.sleep(SLEEP_TIME)
        driver.execute_script("window.scrollBy(0,document.body.scrollHeight)","")
        time.sleep(SLEEP_TIME)
        old_raw = new_raw


    driver.close()
    driver.quit()
    return all_apartments

In [73]:
apartments = get_data_per_page(5)
apartments

Get data from page 5
False
Get data from page 4
True
Get data from page 3
True
Get data from page 2
True


[{'ad_id': 137787704,
  'list_id': 99708659,
  'list_time': None,
  'date': '59 phút trước',
  'account_id': 15981029,
  'account_oid': None,
  'account_name': 'Nguyễn Linh APARTMENT',
  'subject': 'Căn hộ Quận 10_An Ninh Sạch Sẽ_Gần Toà VIETTEL',
  'body': None,
  'category': 1010,
  'category_name': None,
  'area': 105,
  'area_name': 'Quận 10',
  'region': 13,
  'region_name': 'Tp Hồ Chí Minh',
  'company_ad': True,
  'type': 'u',
  'price': 6300000,
  'price_string': '6,3 triệu/tháng',
  'image': 'https://cdn.chotot.com/9QprvKV7dDSDGjy6h53oM5XQZ4t2WCX-z-k690M9Nek/preset:listing/plain/e16dcb5dfedba23056ceeb10f379f422-2801285738674522264.jpg',
  'webp_image': 'https://cdn.chotot.com/o9675yTI2c4JhgtktOuJhD81QKITpfKPykc55nDMsZM/preset:listing/plain/e16dcb5dfedba23056ceeb10f379f422-2801285738674522264.webp',
  'videos': [],
  'special_display_images': ['https://cdn.chotot.com/hDSV00sPxCVlv1BRKhwBaqYFBTTPhZdM4Hd7XoQjtAI/preset:view/plain/aef515c31a5b37fd5e0a7d2c161e6d40-28012857409320327

## 2.4. Filter data

In [61]:
need_keys = ['ad_id', 'list_id', 'subject', 'price', 'size', 'rooms', 'toilets', 'area_name', 'region_name', 'ward_name', 'street_name', 'detail_address', 'escrow_can_deposit', 'zero_deposit', 'longitude', 'latitude']

def filer_apartment(apartment):
    return { key: apartment[key] if key in apartment else '' for key in need_keys }

new_apartments = list(map(filer_apartment, apartments))
new_apartments[0]

{'ad_id': 134359927,
 'list_id': 96900062,
 'subject': '🍀CĂN HỘ 1 PHÒNG NGỦ NGAY ĐH NGÂN HÀNG.SPKT đường 7',
 'price': 6000000,
 'size': 35,
 'rooms': 1,
 'toilets': 1,
 'area_name': 'Thành phố Thủ Đức',
 'region_name': 'Tp Hồ Chí Minh',
 'ward_name': 'Phường Linh Trung (Quận Thủ Đức cũ)',
 'street_name': 'Đường Số 7',
 'detail_address': '',
 'escrow_can_deposit': 2,
 'zero_deposit': False,
 'longitude': 106.76977,
 'latitude': 10.85836}

### 2.5. Write to csv

In [62]:
list_apartment = [[apartment[key] for key in need_keys] for apartment in new_apartments]

with open('./data/hcm-apartment-rent-data.csv', 'w') as f:
    write = csv.writer(f)
    write.writerow(need_keys)
    write.writerows(list_apartment)